In [1]:
from glob import glob
import os
import pandas as pd
import shutil
from pigeon import annotate
import PIL
import matplotlib.pyplot as plt
from IPython.display import display, Image

In [2]:
data_dir = './data_baseline_clean/data_baseline_clean/train'

In [3]:
pattern = data_dir + '/*/*.png'
fps = sorted(glob(pattern))
print(len(fps))

1755


In [4]:
ignore = ['/i/']
fps = [fp for fp in fps if not any([x in fp for x in ignore])]
print(len(fps))

1515


In [5]:
def display_fun(fp):
    num_dir = os.path.basename(os.path.dirname(fp)).upper()
    image = PIL.Image.open(fp)
    image = image.resize((32, 32))
    plt.imshow(image, cmap='binary_r')
    plt.title('{}\n{}'.format(num_dir, os.path.basename(fp)))
    plt.show()
    plt.close()

In [6]:
annotations = annotate(
  fps,
  options=['normal', 'outlier', 'bad_label'],
  display_fn=display_fun
)

HTML(value='0 examples annotated, 1516 examples left')

Output()

In [10]:
df = pd.DataFrame(annotations, columns=['fp', 'label'])
df['outlier'] = (df['label'] == 'outlier')
df['class'] = None
for i, fp in enumerate(df['fp']):
    df.loc[i, 'class'] = os.path.basename(os.path.dirname(fp))

df['outlier'].sum(), df.shape

(157, (1515, 4))

In [17]:
bad = df.loc[(df['label'] == 'bad_label'), 'fp']
for fp in bad:
    dest = fp.replace('/train', '/ignore')
    shutil.move(fp, dest)

In [21]:
df['fp'] = [fp.replace(data_dir, '') for fp in df['fp']]

In [22]:
df

,fp,label,outlier,class
0,/ii/ab2ba65a-ce5d-11eb-b317-38f9d35ea60f.png,bad_label,False,ii
1,/ii/ab2c4722-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,ii
2,/ii/ab2cceae-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,ii
3,/ii/ab2ddcea-ce5d-11eb-b317-38f9d35ea60f.png,outlier,True,ii
4,/ii/ab2e9892-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,ii
...,...,...,...,...
1510,/x/b025799c-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,x
1511,/x/b02616ea-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,x
1512,/x/b0269fa2-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,x
1513,/x/b0276dc4-ce5d-11eb-b317-38f9d35ea60f.png,normal,False,x


In [23]:
df.to_csv('./outliers.csv')